In [2]:
import os
import numpy as np

from bisect import bisect_right, bisect_left

In [3]:
class Rectangle():
    def __init__(self, x, y, width, height):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        
    def GetCentroid(self):
        return (self.x + self.width/2.0, self.y + self.height/2.0)

In [4]:
class UUIDRectangle(Rectangle):
    def __init__(self, x, y, width, height, uuid):
        super().__init__(x, y, width, height)
        self.assigned_to_set = False
        self.uuid = uuid

In [5]:
def PrintMaximalSet(maximal_set):
    for i, rectangle in enumerate(maximal_set):
        if i < len(maximal_set) - 1:
            print("{}, ".format(rectangle.uuid), end = '')
        else: 
            print("{}".format(rectangle.uuid), end = '')

In [6]:
def PrintMaximalSets(maximal_sets):
    for maximal_set in sorted(maximal_sets, key=len, reverse=True):
        PrintMaximalSet(maximal_set)
        print('')

In [8]:
def GetIntersectionMembers(curr, rectangles):
    remainder = rectangles

    # x axis
    remainder.sort(key=lambda r: r.x + r.width)
    list_bc2 = [r.x + r.width for r in remainder] 
    idx = bisect_right(list_bc2, curr.x)
    remainder = remainder[max(0,idx):]

    remainder.sort(key=lambda r: r.x)
    list_bc1 = [r.x for r in remainder] 
    idx = bisect_left(list_bc1, curr.x + curr.width)
    remainder = remainder[0:min(idx, len(remainder))]

    # y axis
    remainder.sort(key=lambda r: r.y + r.height)
    list_bc2 = [r.y + r.height for r in remainder] 
    idx = bisect_right(list_bc2, curr.y)
    remainder = remainder[max(0,idx):]

    remainder.sort(key=lambda r: r.y)
    list_bc1 = [r.y for r in remainder] 
    idx = bisect_left(list_bc1, curr.y + curr.height)
    remainder = remainder[0:min(idx, len(remainder))]
    
    intersection_members = remainder
    
    return intersection_members
    

In [7]:
rectangle_1 = UUIDRectangle(1, 1, 5, 5, 1)
rectangle_2 = UUIDRectangle(4, 4, 3, 3, 2)
rectangle_3 = UUIDRectangle(6, 1, 1, 1, 3)
rectangle_4 = UUIDRectangle(2, 2, 3, 3, 4)
rectangle_5 = UUIDRectangle(1, 4, 1, 3, 5)

rectangles = [rectangle_1, rectangle_2, rectangle_3, rectangle_4, rectangle_5]

In [46]:
def CreateRectanglesFromData():
    rectangles = []
    data = np.loadtxt(os.getcwd() + '/rectangles.txt')

    for i in range(0, data.shape[0]):
        rectangle = UUIDRectangle(data[i,0], data[i,0], data[i,0], data[i,0], i)
        rectangles.append(rectangle)
    
    return rectangles

In [31]:
def BuildGraph(rectangles):
    G = {}
    for r in rectangles:
        G[r.uuid] = []
        intersects = GetIntersectionMembers(r, rectangles)
        for i in intersects:
            if i.uuid == r.uuid: continue
            G[r.uuid].append(i.uuid)
                
    return G

In [83]:
def BK(R, P, X, G, max_sets):
    if len(P) == 0 and len(X) == 0:
        max_sets.append(R)
    
    for v in list(P):
        N_v = set(G[v])
        max_sets = BK(R.union(set({v})), P.intersection(N_v), X.intersection(N_v), G, max_sets)
        P.remove(v)
        X.add(v)
        
    return max_sets

In [84]:
R = set()
X = set()
G = BuildGraph(rectangles)
P = set(G.keys())
max_sets = []

In [85]:
BK(R, P, X, G, max_sets)

[{1, 2, 4}, {1, 5}, {3}]

{5: [1], 4: [1, 2], 1: [4, 5, 2], 2: [1, 4], 3: []}